In [2]:
import pathlib

root = pathlib.Path.cwd().parent

In [11]:
import pandas as pd
import geopandas as gpd
import numpy as np
import statsmodels.api as sm
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances

def load_data(census_filepath, geojson_filepath):
    # Load the census data
    census_data = pd.read_parquet(census_filepath) 
    # Load the GeoJSON data into a GeoDataFrame
    parishes_gdf = gpd.read_file(geojson_filepath)
    
    return census_data, parishes_gdf

def transform_crs(parishes_gdf):
    return parishes_gdf.to_crs(epsg=3006)

def filter_parishes(census_data, parishes_gdf):
    relevant_parishes = census_data['current_parish_ref_code'].unique()
    return parishes_gdf[parishes_gdf['ref_code'].isin(relevant_parishes)]

def merge_data(census_data, parishes_gdf):
    merged_data = census_data.merge(parishes_gdf, left_on="current_parish_ref_code", right_on="ref_code", how="left")
    merged_data = gpd.GeoDataFrame(merged_data, geometry='geometry')
    return merged_data

def compute_centroids(merged_data):
    merged_data['centroid'] = merged_data.geometry.centroid
    return merged_data

def construct_spatial_weights_chunked(merged_data, bandwidth_multiplier=1.0, distance_threshold=None, chunk_size=1000):
    coords = merged_data['centroid'].apply(lambda x: (x.x, x.y)).tolist()
    num_coords = len(coords)
    weights_sparse = csr_matrix((num_coords, num_coords), dtype=np.float64)
    for i in range(0, num_coords, chunk_size):
        end = min(i + chunk_size, num_coords)
        chunk_distances = pairwise_distances(coords[i:end], coords)
        if distance_threshold:
            chunk_distances[chunk_distances > distance_threshold] = 0
        bandwidth = chunk_distances[chunk_distances != 0].mean() * bandwidth_multiplier
        chunk_weights = np.where(chunk_distances != 0, 1 / (chunk_distances / bandwidth), 0)
        weights_sparse[i:end] = csr_matrix(chunk_weights)
    return weights_sparse

def conley_se_regression(merged_data, weights, independent_vars):
    X = merged_data[independent_vars]
    X = sm.add_constant(X)
    y = merged_data['employed']
    model = sm.OLS(y, X).fit()
    residuals = model.resid
    spatial_residuals = weights @ residuals
    spatial_residuals = np.array(spatial_residuals).reshape(-1)
    X_transpose_W = X.T * spatial_residuals
    vcov = X_transpose_W @ X / len(y)
    conley_se = np.sqrt(np.diag(vcov))
    return model.params, conley_se

def main():
    census_filepath = root / "data/census/census_1930.parquet"
    geojson_filepath = root/ "data/maps/Swedish_parishes_1926.geojson"

    census_data, parishes_gdf = load_data(census_filepath, geojson_filepath)
    parishes_gdf = transform_crs(parishes_gdf)
    parishes_gdf = filter_parishes(census_data, parishes_gdf)
    merged_data = merge_data(census_data, parishes_gdf)
    merged_data = compute_centroids(merged_data)
    distance_threshold = 10000  # 10 kilometers
    weights = construct_spatial_weights_chunked(merged_data, bandwidth_multiplier=1.0, distance_threshold=distance_threshold, chunk_size=1000)
    independent_vars = ['birth_parish_treated', 'age', 'age_2', 'female']
    coefficients, conley_se = conley_se_regression(merged_data, weights, independent_vars)
    print("Coefficients:", coefficients)
    print("Conley SE:", conley_se)

if __name__ == "__main__":
    main()

MemoryError: Unable to allocate 20.5 GiB for an array with shape (1000, 2753469) and data type float64

In [12]:
import geopandas as gpd
import pandas as pd
import numpy as np
import statsmodels.api as sm
from shapely.geometry import box

def load_and_filter_data(geojson_path, parquet_path):
    """
    Load the GeoJSON and Excel data, merge them based on the specified columns, 
    and filter the parishes based on the sample.
    
    Args:
    - geojson_path (str): Path to the GeoJSON file.
    - excel_path (str): Path to the Excel file with census data.
    
    Returns:
    - GeoDataFrame: Merged and filtered GeoDataFrame.
    """
    # Load GeoJSON data
    gdf = gpd.read_file(geojson_path)
    
    # Load Excel data
    census_data = pd.read_parquet(parquet_path)
    
    # Join the GeoJSON data with the census data
    merged_gdf = gdf.merge(census_data, left_on='ref_code', right_on='current_parish_ref_code', how='inner')
    
    # Add a constant column for regression
    merged_gdf["const"] = 1
    
    # Filter the GeoDataFrame to only include parishes that appear in the sample
    filtered_gdf = merged_gdf.drop_duplicates(subset=['ref_code'])
    
    return filtered_gdf

def compute_spatial_matrix(filtered_gdf):
    """
    Compute the spatial weighting matrix using queen contiguity.
    
    Args:
    - filtered_gdf (GeoDataFrame): The filtered GeoDataFrame.
    
    Returns:
    - ndarray: Spatial weighting matrix.
    """
    # Compute bounding boxes for each parish
    filtered_gdf['bbox'] = filtered_gdf.geometry.apply(lambda x: box(*x.bounds))
    
    n = len(filtered_gdf)
    W = np.zeros((n, n))
    index_mapping = {idx: i for i, idx in enumerate(filtered_gdf.index)}
    
    # Identify potential neighbors based on bounding box intersections and refine the list
    for idx, row in filtered_gdf.iterrows():
        i = index_mapping[idx]
        potential_neighbors = [index_mapping[potential_idx] for potential_idx in filtered_gdf[filtered_gdf.bbox.intersects(row.bbox)].index]
        for j in potential_neighbors:
            if i != j and (row.geometry.touches(filtered_gdf.loc[filtered_gdf.index[j], 'geometry']) or
                           row.geometry.intersects(filtered_gdf.loc[filtered_gdf.index[j], 'geometry'])):
                W[i, j] = 1
    
    return W

def run_regression_with_conley_se(filtered_gdf, W, outcome_var, independent_var):
    """
    Run a regression and compute Conley standard errors using the spatial weighting matrix.
    
    Args:
    - filtered_gdf (GeoDataFrame): The filtered GeoDataFrame.
    - W (ndarray): Spatial weighting matrix.
    - outcome_var (str): Name of the outcome variable.
    - independent_var (str): Name of the independent variable.
    
    Returns:
    - OLSResults: Regression results.
    - ndarray: Conley standard errors.
    """
    regression_data = filtered_gdf[[outcome_var, independent_var, "const"]]
    
    # Specify the regression model
    model = sm.OLS(regression_data[outcome_var], regression_data[[independent_var, "const"]])
    
    # Fit the model
    results = model.fit()
    
    # Aggregate residuals by parish
    grouped_residuals = filtered_gdf.groupby("ref_code").apply(lambda x: (x[outcome_var] - results.predict(x[[independent_var, "const"]])).sum())
    
    # Calculate the matrix product of aggregated residuals and regression data
    R_aggregated = np.outer(grouped_residuals, regression_data[[independent_var, "const"]].values)
    
    # Compute spatially weighted covariance
    spatial_cov_aggregated = R_aggregated.T @ (W @ R_aggregated)
    
    # Extract the diagonal elements to get the variance for each coefficient
    conley_se_aggregated = np.sqrt(np.diag(spatial_cov_aggregated)[:2] / len(regression_data))
    
    return results, conley_se_aggregated

# Paths to the input files (adjust as needed)
parquet_path = root / "data/census/census_1930.parquet"
geojson_path = root/ "data/maps/Swedish_parishes_1926.geojson"

# Load and filter data
filtered_gdf = load_and_filter_data(geojson_path, parquet_path)

# Compute spatial matrix
W = compute_spatial_matrix(filtered_gdf)

# Run regression and compute Conley SE
outcome_variable = "employed"
independent_variable = "birth_parish_treated"
results, conley_se = run_regression_with_conley_se(filtered_gdf, W, outcome_variable, independent_variable)

print(results.summary2())
print("Conley Standard Errors:", conley_se)

                  Results: Ordinary least squares
Model:                OLS              Adj. R-squared:     0.001    
Dependent Variable:   employed         AIC:                -321.2452
Date:                 2023-10-16 13:50 BIC:                -311.1312
No. Observations:     1161             Log-Likelihood:     162.62   
Df Model:             1                F-statistic:        2.713    
Df Residuals:         1159             Prob (F-statistic): 0.0998   
R-squared:            0.002            Scale:              0.044321 
--------------------------------------------------------------------
                      Coef.  Std.Err.    t     P>|t|   [0.025 0.975]
--------------------------------------------------------------------
birth_parish_treated -0.0483   0.0293  -1.6472 0.0998 -0.1059 0.0092
const                 0.9557   0.0063 151.0451 0.0000  0.9433 0.9682
--------------------------------------------------------------------
Omnibus:              1012.810      Durbin-Watson:   